In [1]:
import pandas as pd
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_csv('../../data/processed/cleaned_reviews.csv')
df.head()

,Unnamed: 0,Sentiment,Time,Text,review_length,clean_reviews,clean_review_length
0,0,positive,2021-06-18,This is a very healthy dog food. Good for thei...,137,healthy dog food good digestion also good smal...,94
1,1,positive,2021-07-07,I've been very pleased with the Natural Balanc...,350,pleased natural balance dog food dogs issues d...,218
2,2,positive,2021-06-18,"Before I was educated about feline nutrition, ...",733,educated feline nutrition allowed cats become ...,508
3,3,positive,2021-07-07,"My holistic vet recommended this, along with a...",493,holistic vet recommended along brands tried ca...,276
4,4,positive,2021-01-07,I bought this coffee because its much cheaper ...,413,bought coffee much cheaper ganocafe organic re...,218


# NMF

In [3]:
vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # keep top 1000 terms 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(df['clean_reviews'])

X.shape # check shape of the document-term matrix

(5444, 1000)

In [12]:
# Create an NMF model
# 10 components will be the topics
nmf_model = NMF(n_components=10, random_state=5)
 
# Fit the model to TF-IDF
nmf_model.fit(X)
 
# Transform the TF-IDF: nmf_features
nmf_features = nmf_model.transform(X)

len(nmf_model.components_) 

10

In [13]:
# Create a DataFrame: components_df
components_df = pd.DataFrame(nmf_model.components_, columns=vectorizer.get_feature_names_out())
components_df

,10,100,12,13,14,15,16,20,24,25,...,worst,worth,wow,wrapped,wrong,year,years,yes,yogurt,yummy
0,0.038679,0.281429,0.147832,0.022089,0.023973,0.054794,0.030988,0.054962,0.080406,0.000294,...,0.000000,0.058153,0.019446,0.001655,0.034818,0.016566,0.016254,0.016555,0.028451,0.005923
1,0.000000,0.000000,0.000000,0.000000,0.008857,0.000000,0.004857,0.005729,0.001738,0.000553,...,0.000810,0.000000,0.011192,0.013444,0.010116,0.019273,0.034965,0.000000,0.000000,0.000000
2,0.006333,0.007867,0.016244,0.000000,0.000000,0.000000,0.001541,0.001411,0.000910,0.000000,...,0.024826,0.019744,0.002272,0.000000,0.022830,0.000000,0.030861,0.006716,0.000022,0.006137
3,0.069512,0.007379,0.083676,0.033413,0.018317,0.035548,0.016032,0.032703,0.043102,0.039177,...,0.000000,0.097793,0.000000,0.009072,0.024427,0.014223,0.056365,0.011661,0.000000,0.005144
4,0.000000,0.009896,0.000460,0.000000,0.000000,0.000000,0.012117,0.000000,0.000000,0.020289,...,0.000000,0.000000,0.011166,0.005928,0.000000,0.025196,0.005388,0.000000,0.011902,0.025597
5,0.022485,0.007014,0.018709,0.011162,0.019626,0.005210,0.000000,0.023965,0.000000,0.000000,...,0.011277,0.020300,0.003008,0.000000,0.020232,0.110097,0.080538,0.037278,0.000579,0.003363
6,0.010919,0.044290,0.024220,0.003095,0.017751,0.031965,0.000686,0.005652,0.007444,0.000000,...,0.000000,0.006419,0.015641,0.004914,0.009732,0.044077,0.029109,0.006528,0.048976,0.054313
7,0.007203,0.000000,0.000000,0.000000,0.000000,0.000000,0.008525,0.000000,0.003438,0.000000,...,0.026380,0.008673,0.005484,0.003786,0.009380,0.016216,0.005464,0.009585,0.008241,0.029766
8,0.021160,0.000000,0.015231,0.000000,0.010835,0.022855,0.007973,0.017123,0.000000,0.000000,...,0.029838,0.055031,0.014945,0.005563,0.021088,0.058074,0.088025,0.000000,0.008480,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005938,0.000000,0.000000,...,0.032975,0.065435,0.003474,0.012473,0.022002,0.000000,0.016276,0.016935,0.000000,0.010124


In [14]:
# Get Words of Highest Value for each Topic
for topic in range(components_df.shape[0]):
    tmp = components_df.iloc[topic]
    print(f'For topic {topic+1} the words with the highest value are:')
    print(tmp.nlargest(10))
    print('\n')

For topic 1 the words with the highest value are:
water      1.714901
drink      1.609790
sugar      1.373425
flavor     0.977873
juice      0.877189
orange     0.822742
soda       0.768531
coconut    0.518884
sweet      0.480891
drinks     0.477744
Name: 0, dtype: float64


For topic 2 the words with the highest value are:
tea       2.758507
green     0.397200
teas      0.242400
iced      0.228202
flavor    0.207818
bags      0.185918
love      0.178159
black     0.169079
drink     0.132433
loose     0.123681
Name: 1, dtype: float64


For topic 3 the words with the highest value are:
coffee      2.350897
cup         0.428861
strong      0.229025
flavor      0.203068
roast       0.195898
coffees     0.179771
flavored    0.160957
beans       0.155292
good        0.154932
smooth      0.142852
Name: 2, dtype: float64


For topic 4 the words with the highest value are:
price       1.503715
amazon      0.843419
store       0.607563
buy         0.553499
local       0.453797
grocery     0.440